# Examples on the Front Page

## Expected Loss

In [2]:
import os
import sys


sys.path.append(os.path.abspath('../'))

## Doc Home Page Example

In [6]:
import quactuary as qa
import quactuary.book as book

from datetime import date
from quactuary.book import (
    LOB, PolicyTerms, Inforce, Portfolio)
from quactuary.distributions.frequency import Poisson
from quactuary.distributions.severity import Exponential
from quactuary.pricing import PricingModel

# Frequency model example
freq = Poisson(mu=3.5)
print(freq.pmf(2))

# Severity model example
sev = Exponential(scale=1000.0)
print(sev.pdf(500.0))

# General Liability Bucket
glpl_policy = PolicyTerms(
    effective_date=date(2026, 1, 1),
    expiration_date=date(2027, 1, 1),
    lob=LOB.GLPL,
    exposure_base=book.SALES,
    exposure_amount=10_000_000_000,
    retention_type="deductible",
    per_occ_retention=1_000_000,
    coverage="occ"
)

glpl_inforce = Inforce(
    n_policies=700,
    terms=glpl_policy,
    frequency=freq,
    severity=sev,
    name = "GLPL 2026 Bucket"
)

portfolio = Portfolio(glpl_inforce)

pricing = PricingModel(portfolio)
result = pricing.simulate()
result.estimates

0.18495897346170082
0.0006065306597126335


{'mean': np.float64(2449951.874110688),
 'variance': np.float64(4860457942.058481),
 'VaR': 2565751.9589995327,
 'TVaR': 2596372.4576630797}

## README.md and http://quactuary.com Examples

In [ ]:
import quactuary as qa
import quactuary.book as book

from datetime import date
from quactuary.backend import set_backend
from quactuary.book import (
    ExposureBase, LOB, PolicyTerms, Inforce, Portfolio)
from quactuary.distributions.frequency import Poisson, NegativeBinomial
from quactuary.distributions.severity import Pareto, Lognormal
from quactuary.pricing import PricingModel


# Workers' Comp Bucket
wc_policy = PolicyTerms(
    effective_date=date(2026, 1, 1),
    expiration_date=date(2027, 1, 1),
    lob=LOB.WC,
    exposure_base=book.PAYROLL,
    exposure_amount=100_000_000,
    retention_type="deductible",
    per_occ_retention=500_000,
    coverage="occ"
)

# General Liability Bucket
glpl_policy = PolicyTerms(
    effective_date=date(2026, 1, 1),
    expiration_date=date(2027, 1, 1),
    lob=LOB.GLPL,
    exposure_base=book.SALES,
    exposure_amount=10_000_000_000,
    retention_type="deductible",
    per_occ_retention=1_000_000,
    coverage="occ"
)

# Frequency-Severity Distributions
wc_freq = Poisson(mu=100)
wc_sev = Pareto(b=1, loc=0, scale=40_000)

glpl_freq = NegativeBinomial(r=50, p=0.5)
glpl_sev = Lognormal(shape=2, loc=0, scale=100_000)

# Book of Business
wc_inforce = Inforce(
    n_policies=1000,
    terms=wc_policy,
    frequency=wc_freq,
    severity=wc_sev,
    name = "WC 2026 Bucket"
)

glpl_inforce = Inforce(
    n_policies=700,
    terms=glpl_policy,
    frequency=glpl_freq,
    severity=glpl_sev,
    name = "GLPL 2026 Bucket"
)

portfolio = wc_inforce + glpl_inforce

pm = PricingModel(portfolio)

# Test using Classical Monte Carlo
set_backend("classical")
classical_result = pm.simulate(n_sims=1_000)
classical_mean = classical_result.estimates["mean"]
print(f"Classical portfolio expected loss: {classical_mean}")

## Risk Measures

In [ ]:
from quactuary.backend import use_backend
from quactuary.distributions.frequency import Geometric
from quactuary.distributions.severity import ContinuousUniformSeverity


# Commercial Auto Bucket
cauto_policy = PolicyTerms(
    effective_date=date(2026, 1, 1),
    expiration_date=date(2027, 1, 1),
    lob=LOB.CAuto,
    exposure_base=book.VEHICLES,
    exposure_amount=50,
    retention_type="deductible",
    per_occ_retention=100_000,
    coverage="occ"
)

# Frequency-Severity Distributions
cauto_freq = Geometric(p=1/8)
cauto_sev = ContinuousUniformSeverity(loc=5_000, scale=90_000)  # scale is width, not upper bound

# Commercial Auto Inforce
cauto_inforce = Inforce(
    n_policies=400,
    terms=cauto_policy,
    frequency=cauto_freq,
    severity=cauto_sev,
    name = "CAuto 2026 Bucket"
)

# Add to Existing Portfolio
portfolio += cauto_inforce
pm2 = PricingModel(portfolio)

# Test using Classical Monte Carlo
with use_backend("classical", num_simulations=1_000):
    classical_result = pm2.simulate(tail_alpha=0.05, n_sims=1_000)
    classical_VaR = classical_result.estimates["VaR"]
    classical_TVaR = classical_result.estimates["TVaR"]
    print(f"Classical portfolio VaR: {classical_VaR}")
    print(f"Classical portfolio TVaR: {classical_TVaR}")